In [1]:
import numpy as np 

from ase.build import bulk 
from forge.workflows.mcmc import MonteCarloAlloySampler
from mace.calculators.mace import MACECalculator 


UMAP not installed. Please install umap-learn to use it, using t-SNE instead


/home/myless/.miniforge3/envs/forge/lib/python3.11/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_indices = torch.lo

In [6]:
from forge.analysis.composition import CompositionAnalyzer
from math import ceil, floor

num_atoms = 2*13**3

x_zr = round(ceil(0.0025 * num_atoms)/num_atoms, 4)
x_cr = round(ceil(0.03 * num_atoms)/num_atoms, 4)
x_ti = round(ceil(0.04 * num_atoms)/num_atoms, 4)
x_w = round(ceil(0.05 * num_atoms)/num_atoms, 4)
x_v = round(1 - x_zr - x_cr - x_ti - x_w, 4)

# if sum of composition is not 1, adjust x_v
if sum(composition.values()) != 1:
    x_v = 1 - sum(composition.values())

composition = {
    'V' : x_v,
    'Cr' : x_cr,
    'Ti' : x_ti,
    'W' : x_w,
    'Zr' : x_zr
}

print(composition)

{'V': 0.8773, 'Cr': 0.03, 'Ti': 0.0401, 'W': 0.0501, 'Zr': 0.0025}


In [8]:
analyzer = CompositionAnalyzer()
atoms = analyzer.create_random_alloy(composition = composition, 
                                         crystal_type = 'bcc', 
                                         dimensions=[13,13,13], 
                                         lattice_constant = 3.01,
                                         balance_element = 'V', 
                                         cubic=True)

print(atoms)

Atoms(symbols='Cr132Ti177V3853W221Zr11', pbc=True, cell=[39.129999999999995, 39.129999999999995, 39.129999999999995])


In [9]:
import forge 

model_path = '../potentials/gen_6_model_0_L0_isolated-2026-01-16_stagetwo.model'

calc = MACECalculator(model_paths=[model_path],
                      device="cuda",
                      default_dtype="float32",
                      enable_cueq=True)

atoms.calc = calc

temperature = 600+273.15
steps_per_atom = 100
total_swaps = steps_per_atom * len(atoms)

mc_sampler = MonteCarloAlloySampler(
    atoms=atoms,
    calculator=calc,
    temperature=temperature,
    steps=total_swaps,
)

final_atoms = mc_sampler.run_mcmc()

/home/myless/.miniforge3/envs/forge/lib/python3.11/site-packages/mace/calculators/mace.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_loc

Converting models to CuEq for acceleration


/home/myless/.miniforge3/envs/forge/lib/python3.11/site-packages/mace/modules/models.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "atomic_numbers", torch.tensor(atomic_numbers, dtype=torch.int64)


KeyboardInterrupt: 